In [2]:
using DSP
using Plots
using LinearAlgebra
plotly();

In [3]:
function lms(x, d, M, mu)
    N = length(x)
    w = zeros(M,N)
    erro = zeros(1,N)
    phi = zeros(M,1)
    ŷ = zeros(1,N)
    for n=1:N-1
        phi = [x[n]; phi[1:M-1]]
        ŷ[n] = (w[:,n])' * phi
        erro[n] = d[n] - ŷ[n]
        w[:,n+1] = w[:,n] + mu*erro[n]*phi
    end
    return w, erro, ŷ
end;

# 1 - Simulação

Crie uma simulação com os parâmetros

- $x = N(0,\sigma_x^2)$
- $v = N(0,\sigma_v^2)$
- $H(z) = 1 - 0.5z^{-1}$

E aplique o LMS

In [4]:
realizacoes = 1_000
N = 700
σx2 = 1
σv2 = 0.01
μ = 0.1

w = zeros(2, N)
erro = zeros(1, N)

h = [1, -0.5];
for i in 1:realizacoes
    x = randn(N)*sqrt(σx2)
    v = randn(N)*sqrt(σv2)
    y = filt(h, x)
    d = y + v
    w_lms, erro_lms = lms(x, d, 2, μ)
    w += w_lms
    erro += erro_lms.^2
end

erro /= realizacoes
w /= realizacoes
;

In [5]:
p1 = plot(w', title = "E{W[n]}", legend = false)
p2 = plot(erro', title = "E{e[n]}", legend = false)
plot(p1, p2, layout = (2,1))

<!DOCTYPE html>
 
 
 Plots.jl

## Valores simulados vs Valores teóricos

Compare os valores obtidos por simulação com os valores teóricos.

In [6]:
Rϕ=[σx2 0;
    0 σx2]

K =[μ*σv2/2 0;
    0 μ*σv2/2]
EMSE = tr(Rϕ*K);
println("Valor teórico de EMSE = $EMSE")

Valor teórico de EMSE = 0.001


In [7]:
plot(title= "Erro quadrático médio")
plot!(erro', label = "Potência simulada de e[n]")
plot!(ones(N)*(σv2 + EMSE), label = "Potência de e[n] teórica")
plot!(ones(N)*σv2, label= "Potência de v[n]")
plot!(ylims = [0.005, 0.02])

<!DOCTYPE html>
 
 
 Plots.jl

No gráfico a cima vemos como o valor simulado de EMSE está próximo do valor teórico

# 2- Filtros de 1 e 3 coeficientes

## Filtro ideal de 1 coeficiente

In [8]:
h = [1];
for i in 1:realizacoes
    x = randn(N)*sqrt(σx2)
    v = randn(N)*sqrt(σv2)
    y = filt(h, x)
    d = y + v
    w_lms, erro_lms = lms(x, d, 2, μ)
    w += w_lms
    erro += erro_lms.^2
end

erro /= realizacoes
w /= realizacoes
;

In [9]:
p1 = plot(w', title = "E{W[n]}", legend = false)
p2 = plot(erro', title = "E{e[n]}", legend = false)
plot(p1, p2, layout = (2,1))

<!DOCTYPE html>
 
 
 Plots.jl

In [10]:
plot(title= "Erro quadrático médio")
plot!(erro', label = "Potência simulada de e[n]")
plot!(ones(N)*(σv2 + EMSE), label = "Potência de e[n] teórica")
plot!(ones(N)*σv2, label= "Potência de v[n]")
plot!(ylims = [0.005, 0.02])

<!DOCTYPE html>
 
 
 Plots.jl

Nos gráficos a cima, vemos que o valor de EMSE continua em torno do valor teórico, e que agora o filtro converge para  um filtro de apenas 1 coeficiente diferente de zero

## Filtro ideal de 3 coeficientes

In [11]:
h = [1, -0.5, 0.2];
for i in 1:realizacoes
    x = randn(N)*sqrt(σx2)
    v = randn(N)*sqrt(σv2)
    y = filt(h, x)
    d = y + v
    w_lms, erro_lms = lms(x, d, 2, μ)
    w += w_lms
    erro += erro_lms.^2
end

erro /= realizacoes
w /= realizacoes
;

In [12]:
p1 = plot(w', title = "E{W[n]}", legend = false)
p2 = plot(erro', title = "E{e[n]}", legend = false)
plot(p1, p2, layout = (2,1))

<!DOCTYPE html>
 
 
 Plots.jl

In [13]:
plot(title= "Erro quadrático médio")
plot!(erro', label = "Potência simulada de e[n]")
plot!(ones(N)*(σv2 + EMSE), label = "Potência de e[n] teórica")
plot!(ones(N)*σv2, label= "Potência de v[n]")
plot!(ylims = [0.005, 0.08])

<!DOCTYPE html>
 
 
 Plots.jl

Nessa situção vemos que o filtro modelado não tinha coeficiente suficiente para se aproximar do filtro ideal e por isso na sua convergência não chegou ao erro em excesso teórico.

## 3 - Filtro LMS de 5 coeficientes

In [14]:
w = zeros(5, N)
erro = zeros(1, N)

h = [1, -0.5];
for i in 1:realizacoes
    x = randn(N)*sqrt(σx2)
    v = randn(N)*sqrt(σv2)
    y = filt(h, x)
    d = y + v
    w_lms, erro_lms = lms(x, d, 5, μ)
    w += w_lms
    erro += erro_lms.^2
end

erro /= realizacoes
w /= realizacoes
;

In [15]:
p1 = plot(w', title = "E{W[n]}", legend = false)
p2 = plot(erro', title = "E{e[n]}", legend = false)
plot(p1, p2, layout = (2,1))

<!DOCTYPE html>
 
 
 Plots.jl

In [16]:
plot(title= "Erro quadrático médio")
plot!(erro', label = "Potência simulada de e[n]")
plot!(ones(N)*(σv2 + EMSE), label = "Potência de e[n] teórica")
plot!(ones(N)*σv2, label= "Potência de v[n]")
plot!(ylims = [0.005, 0.02])

<!DOCTYPE html>
 
 
 Plots.jl

Aqui vemos que o erro em excesso também é maior que o teórico por causa do excesso de coeficientes. Como temos 3 dos 5 coeficientes ondulando próximo de zero, temos a adicção de uma parcela de ruído devido a esses pequenos erros de coeficientes

# 4 - Sinal x diferente de um ruído

### Filtro de 1 coeficiente

In [17]:
M = 2
w = zeros(M, N)
erro = zeros(1, N)

a = 0.9
num_g = [sqrt(1-a^2)]
den_g = [1, -a]
G = PolynomialRatio(num_g, den_g)
h = [1]
μ = 0.05
for i in 1:realizacoes
    u = randn(N)*sqrt(σx2)
    x = filt(G, u)
    v = randn(N)*sqrt(σv2)
    y = filt(h, x)
    d = y + v
    w_lms, erro_lms = lms(x, d, M, μ)
    w += w_lms
    erro += erro_lms.^2
end

erro /= realizacoes
w /= realizacoes
;

In [18]:
p1 = plot(w', title = "E{W[n]}", legend = false)
p2 = plot(erro', title = "E{e[n]}", legend = false)
plot(p1, p2, layout = (2,1))

<!DOCTYPE html>
 
 
 Plots.jl

### Filtro de 2 coeficiente

In [19]:
M = 2
w = zeros(M, N)
erro = zeros(1, N)

a = 0.9
num_g = [sqrt(1-a^2)]
den_g = [1, -a]
G = PolynomialRatio(num_g, den_g)
h = [1, -0.5];
μ = 0.05
for i in 1:realizacoes
    u = randn(N)*sqrt(σx2)
    x = filt(G, u)
    v = randn(N)*sqrt(σv2)
    y = filt(h, x)
    d = y + v
    w_lms, erro_lms = lms(x, d, M, μ)
    w += w_lms
    erro += erro_lms.^2
end

erro /= realizacoes
w /= realizacoes
;

In [20]:
p1 = plot(w', title = "E{W[n]}", legend = false)
p2 = plot(erro', title = "E{e[n]}", legend = false)
plot(p1, p2, layout = (2,1))

<!DOCTYPE html>
 
 
 Plots.jl

### Filtro de 3 coeficiente

In [21]:

M = 2
w = zeros(M, N)
erro = zeros(1, N)

a = 0.9
num_g = [sqrt(1-a^2)]
den_g = [1, -a]
G = PolynomialRatio(num_g, den_g)
h = [1, -0.5, 0.2];
μ = 0.05
for i in 1:realizacoes
    u = randn(N)*sqrt(σx2)
    x = filt(G, u)
    v = randn(N)*sqrt(σv2)
    y = filt(h, x)
    d = y + v
    w_lms, erro_lms = lms(x, d, M, μ)
    w += w_lms
    erro += erro_lms.^2
end

erro /= realizacoes
w /= realizacoes
;

In [22]:
p1 = plot(w', title = "E{W[n]}", legend = false)
p2 = plot(erro', title = "E{e[n]}", legend = false)
plot(p1, p2, layout = (2,1))

<!DOCTYPE html>
 
 
 Plots.jl

Usando um sinal mais complicado que um ruído gaussiano, vemos que o filtro ainda converge para algo próximo do desejado como de esperado